<a href="https://colab.research.google.com/github/vaila-street/py-test/blob/main/20200706_PP_without_reads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Este programa sirve para justificar que unas cuentas de PP sin recargas venían de Vacant

In [ ]:
import pandas as pd 
import numpy as np
import xml.etree.ElementTree as etree

from pathlib import Path
from os.path import join
#home_dir = str(Path.home())
#sys.path.append(join(home_dir, 'pythonclasses'))

#Libería para trabajar con fechas
import datetime
from datetime import date
from datetime import time
from datetime import datetime

#Librería para crear hojas excel
import xlwt
from xlwt.Workbook import *
from pandas import ExcelWriter
import xlsxwriter


import cx_Oracle
import sqlalchemy
from sqlalchemy.engine import create_engine
import time
import glob
import os
import io
import cx_Oracle

from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import Table
from sqlalchemy import Column, String

In [ ]:
def query_rbip(key_entry):
    from sqlalchemy import create_engine
    from sqlalchemy import MetaData
    from sqlalchemy import Table
    from sqlalchemy import Column, String
    #Usuario y claves de la BBDD:
    USERNAME = 'xxxxx' #enter your username
    PASSWORD = 'xxxxxx' #enter your password
    
    #Definimos la conexión de Oracle
    DIALECT = 'oracle'
    SQL_DRIVER = 'cx_oracle'
    HOST = 'ex021-orc.corp.iberdrola.com' #enter the oracle db host url
    PORT = '1540' # enter the oracle port number
    SERVICE = 'RBIP' # enter the oracle db service name
    ENGINE_PATH_WIN_AUTH = f'{DIALECT}+{SQL_DRIVER}://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/?service_name={SERVICE}'
    engine = create_engine(ENGINE_PATH_WIN_AUTH)
    metadata = MetaData()
    #metadata.reflect(bind=engine)
    
    #Creamos la tabla AUX o borramos sus datos si ya existiera
    try:
        aux_table = Table('AUX2', 
                           metadata, 
                           Column('COLUMN1', 
                                  String(80), 
                                  primary_key=True), 
                           schema=USERNAME,
                          )
        aux_table.create(engine)
    except:
        aux_table = metadata.tables[USERNAME+'.AUX2']
        with engine.connect() as conn:
            conn.execute(aux_table.delete()) # Vaciamos la tabla 

    
    #Carga los datos en la columna de la tabla auxiliar
    column_to_upload = key_entry.iloc[:,0] # The data that want to upload to the database
    clean_data = [str(i) for i in list(set(column_to_upload))] # The data must be uploaded as string without duplicates
    data = [{'COLUMN1':str(i)} for i in clean_data if i==i]

    with engine.connect() as conn:
        conn.execute(aux_table.insert(), data)
    #with engine.connect() as conn:
        result_q1 = conn.execute(sql_query1)
        output_q1 = result_q1.fetchall()

    aux_table.drop(engine) # Eliminamos la AUX
    engine.dispose() # Cerramos la conexión de DB
    
    return pd.DataFrame(output_q1, columns=result_q1.keys())

In [ ]:
def query_bw(key_entry):
    from sqlalchemy import create_engine
    from sqlalchemy import MetaData
    from sqlalchemy import Table
    from sqlalchemy import Column, String
    #Usuario y claves de la BBDD:
    USERNAME = 'xxxx' #enter your username
    PASSWORD = 'xxxx' #enter your password
        
    #Definimos la conexión de Oracle
    DIALECT = 'oracle'
    SQL_DRIVER = 'cx_oracle'
    HOST = 'ex021-orc.corp.iberdrola.com' #enter the oracle db host url
    PORT = '1540' # enter the oracle port number
    SERVICE = 'BWUP' # enter the oracle db service name
    ENGINE_PATH_WIN_AUTH = f'{DIALECT}+{SQL_DRIVER}://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/?service_name={SERVICE}'
    engine = create_engine(ENGINE_PATH_WIN_AUTH)
    metadata = MetaData()
    #metadata.reflect(bind=engine)
    
    #Creamos la tabla AUX o borramos sus datos si ya existiera
    try:
        aux_table = Table('U406403_AUX2', 
                           metadata, 
                           Column('COLUMN1', 
                                  String(80), 
                                  primary_key=True), 
                           schema=USERNAME,
                          )
        aux_table.create(engine)
    except:
        aux_table = metadata.tables[USERNAME+'.U406403_AUX2']
        with engine.connect() as conn:
            conn.execute(aux_table.delete()) # Vaciamos la tabla 

    
    #Carga los datos en la columna de la tabla auxiliar
    column_to_upload = key_entry.iloc[:,0] # The data that want to upload to the database
    clean_data = [str(i) for i in list(set(column_to_upload))] # The data must be uploaded as string without duplicates
    data = [{'COLUMN1':str(i)} for i in clean_data if i==i]

    with engine.connect() as conn:
        conn.execute(aux_table.insert(), data)
    #with engine.connect() as conn:
        result_q1 = conn.execute(sql_query1(USERNAME))
        output_q1 = result_q1.fetchall()

    aux_table.drop(engine) # Eliminamos la AUX
    engine.dispose() # Cerramos la conexión de DB
    
    return pd.DataFrame(output_q1, columns=result_q1.keys())

In [ ]:
import numpy as np
import datetime
today = np.datetime64(datetime.datetime.now())


In [ ]:
USERNAME = 'U406403'
USERNAME_BW = 'U_UKBW_001'

In [ ]:
#Indicamos la ruta de trabajo
import os
workfolder=r'C:\Users\U406403\Desktop\Python\20210705_AW_PP_without_reads' 
os.chdir(workfolder) 

In [ ]:
path_log = r'C:\Users\U406403\Desktop\Python\20210705_AW_PP_without_reads\Robot.log' 
def writelog(text, file=path_log): 
    from datetime import datetime 
    with open(file, 'a') as file: 
        now = datetime.today().strftime('%Y-%m-%d %H:%M:%S') 
        file.write(now + ' ' + text +'\n') 

In [ ]:
#Cargamos el archivo de partida
file = 'PP_Export.xlsx' 
raw = pd.read_excel(join(workfolder,file), sheet_name='Exportar Hoja de Trabajo')

In [ ]:
#Formateamos el dataframe
raw.columns= raw.columns.str.lower()
raw = raw.rename(columns={'contrac_acc':'ca',})
raw['ca']=raw['ca'].fillna(0).astype(np.int64).apply(str).str.zfill(12)

In [ ]:
key_entry = raw[['installation']]

In [ ]:
#Lanzamos la query para obtener el POD
def sql_query1(username): return """
   SELECT 		
		b..anlage  AS installation		
        a.EXT_UI  AS POD		
    FROM 		
    	cdc.UAP_EUITRANS@rbip a    	
    INNER JOIN 		
        cdc.UAP_EUIINSTLN@rbip b		
        	ON a.int_UI = b.INT_UI	
    INNER JOIN
        %s.U406403_AUX2 tb
        ON 
        tb.COLUMN1 = b.ANLAGE
                """%username

In [ ]:
#Lanzamos la query para obtener el POD
def sql_query1(username): return """
    SELECT
        ltrim(a.mpxn,0) AS pod,
        a.installation
    FROM reta.mv_integrated_md@rbip a
    INNER JOIN
        %s.U406403_AUX2 tb
        ON 
        tb.COLUMN1 = a.installation
                """%username

In [ ]:
writelog('query1_start')
df_accounts1 = query_bw(key_entry)
writelog('query1_end')

In [ ]:
#Incluimos el POD
df_accounts1.drop_duplicates(subset=['installation'], inplace=True)
df_accounts1['installation']= df_accounts1['installation'].astype(np.int64)
raw = pd.merge(raw, df_accounts1, how= 'left', on='installation')

In [ ]:
df_accounts1[df_accounts1['installation']==3000004096]

,pod,installation
55612,1273287509,3000004096


In [ ]:
key_entry = raw[['pod']]

In [ ]:
#Lanzamos la query para obtener el POD
def sql_query1(username): return """
    SELECT 
        ltrim(a.ext_UI,0) AS pod,
        a.z_date AS action_date,
        a.ABLHINW AS mr_note
    FROM
        cdc.UAP_ZUDM_TMRFREETEXT@rbip a
    INNER JOIN
        %s.U406403_AUX2 tb
        ON 
        tb.COLUMN1 = ltrim(a.ext_ui,0)
                """%username

In [ ]:
writelog('query2_start')
df_accounts2 = query_bw(key_entry)
writelog('query2_end')

In [ ]:
file = 'Free text codes.xlsx'
Codes_mapping = pd.read_excel(join(workfolder,file), sheet_name='Sheet1')
Codes_mapping['MR_NOTE'] = Codes_mapping['MR_NOTE'].astype(str)
Codes_mapping = Codes_mapping.rename(columns={'MR_NOTE':'mr_note',})

#Ponemos la MR_NOTE en texto para evitar problemas y borramos la fecha de los casos con MR_NOTE vacío
df_accounts2['mr_note'] = df_accounts2['mr_note'].astype(str)
df_accounts2['mr_note'] = df_accounts2['mr_note'].str.replace(' ', '')

df_accounts2['action_date'] = np.where(df_accounts2['mr_note']=='', '', df_accounts2['action_date'])

df_accounts2 = pd.merge(df_accounts2, Codes_mapping, how='left', on='mr_note' )


In [ ]:
df_accounts2.head()

,pod,action_date,mr_note,Freetext_Description,Freetext_Summary,Freetext_Summary_2
0,5067442101,20170110,E133,Premises not visited,Others,No Contact
1,1001027202,20180913,E128,Unable to access premises,Others,No Contact
2,1001144700,20121018,E128,Unable to access premises,Others,No Contact
3,1001144700,20130117,E169,Insufficient address,Possible addess query,No Contact
4,1001144700,20131015,E169,Insufficient address,Possible addess query,No Contact


In [ ]:
midate = raw[['pod', 'contract_mi']]
midate.drop_duplicates(subset=['pod'], inplace=True)

df_accounts2 = pd.merge(df_accounts2, midate, how='left', on='pod' )

df_accounts2['action_date'] = pd.to_datetime(df_accounts2['action_date'],format='%Y%m%d',errors='coerce')
df_accounts2['contract_mi'] = pd.to_datetime(df_accounts2['contract_mi'],format='%Y%m%d',errors='coerce')


df_accounts2 = df_accounts2[(df_accounts2['contract_mi']>df_accounts2['action_date'])]
df_accounts2 = df_accounts2.sort_values(['pod','action_date'],ascending=[True, False])
df_accounts2.drop_duplicates(subset=['pod'], inplace=True)
df_accounts2 = df_accounts2.drop(['contract_mi'], axis=1)

raw = pd.merge(raw, df_accounts2, how='left', on='pod' )

C:\Users\U406403\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
key_entry = raw[['installation']]

In [ ]:
#Lanzamos la query para obtener la CA anterior a PP
def sql_query1(username): return """
    WITH aux as(
    SELECT
        a.installation,
        a.contract_account,
    row_number() OVER (PARTITION BY a.installation ORDER BY a.moveout_date desc) RN
    FROM 
        reta.mv_integrated_md@rbip a
    INNER JOIN
        %s.U406403_AUX2 tb
        ON 
        tb.COLUMN1 = a.installation
    WHERE
        payment_plan != 'BMW'
    ORDER BY moveout_date desc
    )
    SELECT
        installation,
        contract_account as old_ca
    FROM AUX
    WHERE rn = 1
                """%username

In [ ]:
writelog('query3_start')
df_accounts3 = query_bw(key_entry)
writelog('query3_end')

In [ ]:
df_accounts3['installation']= df_accounts3['installation'].astype(np.int64)
df_accounts3.drop_duplicates(subset=['installation'], inplace=True)
raw = pd.merge(raw, df_accounts3, how='left', on='installation' )

In [ ]:
key_entry = raw[['old_ca']]

In [ ]:
#Lanzamos la query para obtener el último maintin
def sql_query1(username): return """
    SELECT
        a.vkont as old_ca,
        a.zzcode as maintin,
        a.zzmfvdat as maintin_date
    FROM cdc.UAP_ZUFI_T_PLACEHIST@rbip a
    JOIN 
        dunning_config b
    ON 
        a.zzcode = b.value_enabled
    INNER JOIN
        %s.U406403_AUX2 tb
        ON 
        tb.COLUMN1 = a.vkont
    WHERE 
        a.zzrectyp = 'I'
    AND a.zzplace_typ != 'WARRANT'
    AND a.zzplace_typ != 'PPM WARRANT'
    AND 
        b.field = 'OUTCOME_ON_LAST_MONTHS'
                """%username

In [ ]:
writelog('query4_start')
df_accounts4 = query_bw(key_entry)
writelog('query4_end')

In [ ]:
df_accounts4['maintin_date'] = pd.to_datetime(df_accounts4['maintin_date'],format='%Y%m%d',errors='coerce')
raw['contract_mi'] = pd.to_datetime(raw['contract_mi'],format='%Y%m%d',errors='coerce')
midate = raw[['old_ca', 'contract_mi']]
midate.drop_duplicates(subset=['old_ca'], inplace=True)
df_accounts4 = pd.merge(df_accounts4, midate, how='left', on='old_ca' )


df_accounts4 = df_accounts4[(df_accounts4['contract_mi']>df_accounts4['maintin_date'])]
df_accounts4 = df_accounts4.sort_values(['old_ca','maintin_date'],ascending=[True, False])
df_accounts4.drop_duplicates(subset=['old_ca'], inplace=True)
df_accounts4 = df_accounts4.drop(['contract_mi'], axis=1)

C:\Users\U406403\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [ ]:
raw = pd.merge(raw, df_accounts4, how='left', on='old_ca' )

In [ ]:
raw = raw.rename(columns={'old_ca':'last_ca_noBMW',})

In [ ]:
key_entry = raw[['installation']]

In [ ]:
#Lanzamos la query para obtener la CA anterior
def sql_query1(username): return """
    SELECT
        a.installation,
        a.contract_account as last_ca,
        a.payment_plan as last_ca_payplan,
        a.moveout_date as last_ca_mo
    FROM 
        reta.mv_integrated_md@rbip a
    INNER JOIN
        %s.U406403_AUX2 tb
        ON 
        tb.COLUMN1 = a.installation
                """%username

In [ ]:
writelog('query5_start')
df_accounts5 = query_bw(key_entry)
writelog('query5_end')

In [ ]:
df_accounts5.head()

,installation,last_ca,last_ca_payplan,last_ca_mo
0,3003940700,061891874021,BMW,2020-02-05 00:00:00
1,3005048005,060211985029,BMW,2016-08-18 00:00:00
2,3005048005,060211985029,BMW,9999-12-31 00:00:00
3,3005270932,016082174238,BMW,9999-12-31 00:00:00
4,3003961072,016072972712,BMW,9999-12-31 00:00:00


In [ ]:
# raw = raw.drop(['last_ca'], axis=1)
# raw = raw.drop(['last_ca_payplan'], axis=1)
# raw = raw.drop(['moveout_date'], axis=1)


In [ ]:
#Todos los casos de 9999 los pasamo a 2200 para que no de error al pasar a fecha
df_accounts5['last_ca_mo'] = df_accounts5['last_ca_mo'].astype(str)
df_accounts5['last_ca_mo']= df_accounts5['last_ca_mo'].str.replace('9999', '2200')
df_accounts5['last_ca_mo'] = pd.to_datetime(df_accounts5['last_ca_mo'],format='%Y-%m-%d 00:00:00',errors='coerce')

#Ordenamos de mayor a menor
df_accounts5 = df_accounts5.sort_values(['last_ca','last_ca_mo'],ascending=[True, False])

In [ ]:
#Comparamos la fecha contra la de la CA y nos quedamos justo con la anterior ca.
midate = raw[['installation', 'contract_mi']]
midate.drop_duplicates(subset=['installation'], inplace=True)
df_accounts5['installation']= df_accounts5['installation'].astype(np.int64)
df_accounts5 = pd.merge(df_accounts5, midate, how= 'left', on='installation')

df_accounts5 = df_accounts5[(df_accounts5['contract_mi']>=df_accounts5['last_ca_mo'])]
df_accounts5 = df_accounts5.sort_values(['last_ca','last_ca_mo'],ascending=[True, False])
df_accounts5.drop_duplicates(subset=['installation'], inplace=True)
df_accounts5 = df_accounts5.drop(['contract_mi'], axis=1)

raw = pd.merge(raw, df_accounts5, how = 'left', on='installation')

C:\Users\U406403\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
raw.head()

,installation,contract,div,mi_year,mi_year_comment,mi_year_comment_2,contract_mi,contract_mo,ca,zzpayment_plan,...,mr_note,Freetext_Description,Freetext_Summary,Freetext_Summary_2,last_ca_noBMW,maintin,maintin_date,last_ca,last_ca_payplan,last_ca_mo
0,3000004096,62704402,2,2016,2016,2016,2016-04-26,99991231,016061945783,BMW,...,E128,Unable to access premises,Others,No Contact,016020092500,NaN,NaT,016020092500,RCQ,2016-04-25
1,3000007223,66125098,1,2019,2019,2019,2019-11-25,99991231,016088707077,BMW,...,20,No Access,Others,No Contact,016042935167,52,2019-08-05,009076453015,BDM,2016-04-30
2,3000009773,60864734,1,2017,2017,2017,2017-02-20,99991231,016046109190,BMW,...,20,No Access,Others,No Contact,090163356013,NaN,NaT,090163356013,RCQ,2017-02-19
3,3000011470,62643155,1,2018,2018,2018,2018-01-08,99991231,016061410774,BMW,...,19,Call not made on routine visit,Others,No Contact,016044686144,52,2017-10-13,016000049617,BCM,2016-03-16
4,3000013813,64537002,2,2019,2019,2019,2019-06-06,99991231,016077584175,BMW,...,E133,Premises not visited,Others,No Contact,016026845787,NaN,NaT,016025074149,RCQ,2016-01-31


In [ ]:
#Creamos el scenario1
raw.loc[(raw['last_ca_noBMW']==raw['last_ca']),'Scenario_1'] = 'Last CA was BMW'
raw.loc[(raw['last_ca'].isnull()),'Scenario_1'] = 'No previous CA'
raw.loc[(raw['last_ca_noBMW'].isnull()),'Scenario_1'] = 'Only BMW CAs previously'
raw.loc[(raw['Scenario_1'].isnull()),'Scenario_1'] = 'Last CA was no BMW'

In [ ]:
#Creamos el scenario2
raw.loc[(raw['maintin']=='52'),'Scenario_DCA'] = 'No contact3rd'
raw.loc[(raw['maintin']=='59'),'Scenario_DCA'] = 'No contact'
raw.loc[(raw['maintin']=='49'),'Scenario_DCA'] = 'Vacant'
raw.loc[(raw['maintin'].isnull()),'Scenario_DCA'] = 'No previous maintin'
raw.loc[(raw['Scenario_DCA'].isnull()),'Scenario_DCA'] = 'Customer contacted'

In [ ]:
#Creamos el scenario3
raw.loc[(raw['Freetext_Summary_2']=='No Contact'),'Scenario_DR'] = 'Last DR visit pre contract with no contact'
raw.loc[(raw['mr_note']=='E126'),'Scenario_DR'] = 'Last DR visit pre contract with Vacant'
raw.loc[(raw['Freetext_Summary_2']=='Contact'),'Scenario_DR'] = 'Last DR visit pre contract with contact'
raw.loc[(raw['Freetext_Summary_2'].isnull()),'Scenario_DR'] = 'No previous DR visit'


In [ ]:
raw['contract'] = raw['contract'].apply(str).str.zfill(10)
key_entry = raw[['contract']]


In [ ]:
raw.head()

,installation,contract,div,mi_year,mi_year_comment,mi_year_comment_2,contract_mi,contract_mo,ca,zzpayment_plan,...,Freetext_Summary_2,last_ca_noBMW,maintin,maintin_date,last_ca,last_ca_payplan,last_ca_mo,Scenario_1,Scenario_DCA,Scenario_DR
0,3000004096,0062704402,2,2016,2016,2016,2016-04-26,99991231,016061945783,BMW,...,No Contact,016020092500,NaN,NaT,016020092500,RCQ,2016-04-25,Last CA was BMW,No previous maintin,Last DR visit pre contract with no contact
1,3000007223,0066125098,1,2019,2019,2019,2019-11-25,99991231,016088707077,BMW,...,No Contact,016042935167,52,2019-08-05,009076453015,BDM,2016-04-30,Last CA was no BMW,No contact3rd,Last DR visit pre contract with no contact
2,3000009773,0060864734,1,2017,2017,2017,2017-02-20,99991231,016046109190,BMW,...,No Contact,090163356013,NaN,NaT,090163356013,RCQ,2017-02-19,Last CA was BMW,No previous maintin,Last DR visit pre contract with no contact
3,3000011470,0062643155,1,2018,2018,2018,2018-01-08,99991231,016061410774,BMW,...,No Contact,016044686144,52,2017-10-13,016000049617,BCM,2016-03-16,Last CA was no BMW,No contact3rd,Last DR visit pre contract with no contact
4,3000013813,0064537002,2,2019,2019,2019,2019-06-06,99991231,016077584175,BMW,...,No Contact,016026845787,NaN,NaT,016025074149,RCQ,2016-01-31,Last CA was no BMW,No previous maintin,Last DR visit pre contract with no contact


In [ ]:
#Lanzamos la query para obtener la CA anterior
def sql_query1(username): return """
    SELECT
        a.contract,
        a.contract_status
    FROM 
        reta.mv_integrated_md@rbip a
    INNER JOIN
        %s.U406403_AUX2 tb
        ON 
        tb.COLUMN1 = a.contract
                """%username

In [ ]:
writelog('query6_start')
df_accounts6 = query_bw(key_entry)
writelog('query6_end')

In [ ]:
df_accounts6.drop_duplicates(subset=['contract'], inplace=True)

raw = pd.merge(raw, df_accounts6, how = 'left', on='contract')

In [ ]:
PP_report_raw[PP_report_raw['contract']=='67836368']
PP_file

'Report_Deuda_Meters_20210706.xlsx'

In [ ]:
#Vamos a cargar la info del PP tracker
PP_tracker_path = r'\\Cathnas01\FIS Project\24 Analysis & Design\Debt\Reporting\GBO debt cleanse campaigns\PP tracker'

PP_file = 'Report_Deuda_Meters_2021'
PP_files =[]
os.chdir(PP_tracker_path)
for file in glob.glob(PP_file+'*'):
    PP_files.append(file)
PP_files = [i for i in PP_files if i[-4:]=='xlsx']
PP_files.sort(reverse=True)
PP_file = PP_files[0]

#Cargamos el archivo seleccionado
PP_report_raw = pd.read_excel(PP_tracker_path+'/'+PP_file, sheet_name='DATA')[['CONCTRACT_INT','Scenario NP','Scenario']]

PP_report_raw = PP_report_raw.rename(columns={'CONCTRACT_INT':'contract',
                                             'Scenario':'Scenario_PP'})

PP_report_raw['contract'] = PP_report_raw['contract'].astype(np.int64)
PP_report_raw['contract'] = PP_report_raw['contract'].apply(str).str.zfill(10)
raw = pd.merge(raw, PP_report_raw, how = 'left', on='contract')

In [ ]:
key_entry = raw[['pod']]

In [ ]:
#Lanzamos la query para obtener el POD
def sql_query1(username): return """
    SELECT 
        ltrim(a.ext_UI,0) AS pod,
        a.z_date AS action_date,
        a.ABLHINW AS mr_note
    FROM
        cdc.UAP_ZUDM_TMRFREETEXT@rbip a
    INNER JOIN
        %s.U406403_AUX2 tb
        ON 
        tb.COLUMN1 = ltrim(a.ext_ui,0)
                """%username

In [ ]:
writelog('query7_start')
df_accounts7 = query_bw(key_entry)
writelog('query7_end')

In [ ]:
# file = 'Free text codes.xlsx'
# Codes_mapping = pd.read_excel(join(workfolder,file), sheet_name='Sheet1')
# Codes_mapping['MR_NOTE'] = Codes_mapping['MR_NOTE'].astype(str)
# Codes_mapping = Codes_mapping.rename(columns={'MR_NOTE':'mr_note',})

#Ponemos la MR_NOTE en texto para evitar problemas y borramos la fecha de los casos con MR_NOTE vacío
df_accounts7['mr_note'] = df_accounts7['mr_note'].astype(str)
df_accounts7['mr_note'] = df_accounts7['mr_note'].str.replace(' ', '')

df_accounts7['action_date'] = np.where(df_accounts7['mr_note']=='', '', df_accounts7['action_date'])

df_accounts7 = pd.merge(df_accounts7, Codes_mapping, how='left', on='mr_note' )


In [ ]:
midate = raw[['pod', 'contract_mi']]
midate.drop_duplicates(subset=['pod'], inplace=True)

df_accounts7 = pd.merge(df_accounts7, midate, how='left', on='pod' )

df_accounts7['action_date'] = pd.to_datetime(df_accounts7['action_date'],format='%Y%m%d',errors='coerce')
#df_accounts2['contract_mi'] = pd.to_datetime(df_accounts2['contract_mi'],format='%Y%m%d',errors='coerce')


df_accounts7a = df_accounts7[(df_accounts7['contract_mi']<=df_accounts7['action_date'])&
                           (df_accounts7['mr_note']=='E126')]
df_accounts7a = df_accounts7a.sort_values(['pod','action_date'],ascending=[True, False])
df_accounts7a.drop_duplicates(subset=['pod'], inplace=True)
df_accounts7a = df_accounts7a[['pod','mr_note']]
df_accounts7a['mr_note'] = 'Vacant code received'
df_accounts7a = df_accounts7a.rename(columns={'mr_note':'DR_Vacant'})



df_accounts7b = df_accounts7[(df_accounts7['contract_mi']<=df_accounts7['action_date'])]
df_accounts7b = df_accounts7b.sort_values(['pod','action_date'],ascending=[True, False])
df_accounts7b.drop_duplicates(subset=['pod'], inplace=True)
df_accounts7b = df_accounts7b.drop(['contract_mi'], axis=1)
df_accounts7b = df_accounts7b.rename(columns={'mr_note':'last_mr_note',
                                             'action_date':'last_action_date',
                                             'Freetext_Description':'Last_Freetext_Description',
                                             'Freetext_Summary':'Last_Freetext_Summary',
                                             'Freetext_Summary_2':'Last_Freetext_Summary_2'})


raw = pd.merge(raw, df_accounts7a, how='left', on='pod' )
raw = pd.merge(raw, df_accounts7b, how='left', on='pod' )

C:\Users\U406403\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
df_accounts7.head()

,pod,action_date,mr_note,Freetext_Description,Freetext_Summary,Freetext_Summary_2,contract_mi
0,1010714604,2020-06-01,E128,Unable to access premises,Others,No Contact,2019-06-25
1,1011011103,2013-07-04,E128,Unable to access premises,Others,No Contact,2016-04-03
2,1011011103,2013-10-03,E128,Unable to access premises,Others,No Contact,2016-04-03
3,1011011103,2014-03-28,E128,Unable to access premises,Others,No Contact,2016-04-03
4,1011011103,2015-07-12,E128,Unable to access premises,Others,No Contact,2016-04-03


In [ ]:
key_entry = raw[['installation']]

In [ ]:
#Lanzamos la query para obtener el MSN y material
def sql_query1(username): return """
     SELECT 
        a.ANLAGE    AS installation,
        c.SERGE AS msn
        --c.MATNR AS material
    FROM 
        cdc.UAP_EASTS@rbip a
    INNER JOIN 
        cdc.UAP_ETDZ@rbip b 
            ON a.LOGIKZW = b.LOGIKZW 
    INNER JOIN 
        cdc.UAP_EQUI@rbip c 
            ON b.EQUNR =c.EQUNR 
    INNER JOIN
        %s.U406403_AUX2 tb
        ON 
        tb.COLUMN1 = a.ANLAGE
    WHERE 
            a.BIS = '99991231'
        AND 
            b.BIS = '99991231'
        AND
            a.ZWNSETTL <> 'X' 
                """%username

In [ ]:
writelog('query8_start')
df_accounts8 = query_bw(key_entry)
writelog('query8_end')

In [ ]:
#Incluimos la info del MSN y Material
df_accounts8['installation']=df_accounts8['installation'].astype(np.int64)
df_accounts8.drop_duplicates(subset=['installation'], inplace=True)
raw = pd.merge(raw, df_accounts8, how='left', on='installation')

In [ ]:
key_entry = raw[['contract']]

In [ ]:
#Lanzamos la query para obtener el MSN y material
def sql_query1(username): return """
    SELECT
        a.vertrag as contract,
        a.CONT4_ERROR 
    FROM cdc.UAP_ZISU_COH_CONTR@rbip a
    INNER JOIN
        %s.U406403_AUX2 tb
        ON 
        tb.COLUMN1 = a.vertrag
                """%username

In [ ]:
writelog('query9_start')
df_accounts9 = query_bw(key_entry)
writelog('query9_end')

In [ ]:
#Incluimos la info del MSN y Material
df_accounts9.drop_duplicates(subset=['contract'], inplace=True)
raw = pd.merge(raw, df_accounts9, how='left', on='contract')

In [ ]:
#Vamos a cargar la info del report de missalignments de Scott
Scott_path = r'C:\Users\U406403\Desktop\Python\20210705_AW_PP_without_reads'
Scott_file = 'PPMIP_Scott.xlsx'

#Cargamos el archivo seleccionado
Scott_report_gas = pd.read_excel(join(Scott_path,Scott_file), sheet_name='Gas')[['installation','Category_MSN_miss']]
Scott_report_elec = pd.read_excel(join(Scott_path,Scott_file), sheet_name='Electricity')[['installation','Category_MSN_miss']]

Scott_report = Scott_report_gas.append(Scott_report_elec)

Scott_report.drop_duplicates(subset=['installation'], inplace=True)
raw = pd.merge(raw, Scott_report, how = 'left', on='installation')


In [ ]:
raw = raw.rename(columns={'div':'divis','zzpayment_plan':'payplan'})

In [ ]:
raw.to_excel(join(workfolder,'PP_without_reads.xlsx'),'DATA',index=False)

In [ ]:
# raw = raw.drop(['resumen'], axis=1)
# raw = raw.drop(['resumen_post'], axis=1)
# raw = raw.drop(['current_agency'], axis=1)
# raw = raw.drop(['agency_summary'], axis=1)
# raw = raw.drop(['exclusion'], axis=1)

In [ ]:
#raw = raw.drop(['pod'], axis=1)